# Create dataframe of master burst coordinates for future use
### Master burst coordinates created in ArcGIS and attribute table details stored in a text file

In [ ]:
# developed 15 Feb 2019, S Lawrie, GA

import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import cv2

%matplotlib inline 
# required for correct plotting in jupyter notebook

### Orientation Details

In [ ]:
burst_dir = "/g/data1/dg9/SENTINEL-1_BURSTS"

#orient = "Ascending"
orient = "Descending"

#input_file = "S1_master_bursts_ascending.txt"
input_file = "S1_master_bursts_descending.txt"

In [ ]:
def master_coords():
    "Creates a dataframe with shapely polygons of master burst coordinates which have a buffer applied." 
    "To save future processing time, the dataframe is saved for use when creating 'frame' scene stacks."
    
    # load master burst coordinates text file into a dataframe 
    df_master_input = pd.read_csv("%s/MASTER_BURSTS/%s" %(burst_dir,input_file), sep=",")

    ## Convert coordinates into shapely polygons and put into new dataframe
    df_master = pd.DataFrame([])  # create blank dataframe

    # iterate over coordinates and append to blank dataframe
    for i, row in df_master_input.iterrows():
        mission = row[1]
        mode = row[2]
        type1 = row[3]
        pass1 = row[4]
        orbit = row[5]
        frame = row[6]        
        burst_num = row[7]  
        swath = row[8] 
        polar = row[9]
        ul = Point(float(row[10]), float(row[11]))
        ur = Point(float(row[12]), float(row[13]))
        lr = Point(float(row[14]), float(row[15]))
        ll = Point(float(row[16]), float(row[17]))
        pointList = [ul, ur, lr, ll]
        poly = Polygon([[p.x, p.y] for p in pointList]) # creates shapely polygon
        df_temp1 =  gpd.GeoDataFrame([[mission,type1,mode,pass1,orbit,frame,burst_num,swath,polar,poly]],
                          columns = ['Mission','Type','Mode','Pass','RelOrbit','Frame','BurstNum','Swath','Polar','Extent'],
                          geometry='Extent')
        df_master = df_master.append(df_temp1, ignore_index=True)
    
    # calculate buffer for each master burst extent and add to dataframe
    for i in df_master.index:
        poly = df_master.at[i,'Extent']
        buffer = poly.buffer(+0.02, cap_style=2,join_style=2)
        df_master.at[i,'Buffer'] = buffer
    
    # save master burst coordinates dataframe for future use
    name = '%s/MASTER_BURSTS/S1_IW_SLC_%s_Master_Bursts_Dataframe' %(burst_dir,orient)
    df = df_master.to_pickle(name)
    
    return

### Run Function

In [ ]:
master_coords()

#### check dataframe result

In [ ]:
name = '%s/MASTER_BURSTS/S1_IW_SLC_%s_Master_Bursts_Dataframe' %(burst_dir,orient)
df_master = pd.read_pickle(name)
df_master.head()